# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Análisis exploratorio y curación de datos

### Trabajo práctico entregable - Grupo 22 - Parte 1

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago
- Spano, Marcelo

**Seguimiento:** Meinardi, Vanesa

---

## Librerías

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine, text

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None  # default='warn'

sns.set_context('talk')
sns.set_theme(style='white')

## Acerca de los datasets

El dataset a utilizar proviene de la [compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia. Particularmente, utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Este [dataset](https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv) está disponible en internet, desde donde lo usaremos.

Por otro lado, vamos a aumentar los datos presentes en dicho conjunto utilizando un dataset similar: las publicaciones de la plataforma AirBnB en Melbourne en el año 2018. El objetivo es estimar con mayor precisión el valor del vecindario de cada propiedad. Este otro [dataset](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv), también disponible en internet, es un conjunto de datos de *scrapings* del sitio realizado por [Tyler Xie](https://www.kaggle.com/tylerx), también disponible en una competencia de Kaggle.

---
# Ejercicio 1 - SQL

## 1) Conexión

Para poder ejecutar consultas simples en SQL con SQLAlchemy, primero debemos crear un ***engine*** : es el punto de partida para cualquier aplicación que hagamos de SQLAlchemy, proporcionando una forma de conectarse e interactuar con la base de datos. El mismo provee además:
- Una ***connection pool***: conjunto de conexiones a la base de datos que permanecen activas por largos períodos de tiempo y se pueden reutilizar eficientemente, previniendo el *overhead* que deviene de la creación de nuevas conexiones, y aumentando la velocidad de funcionamiento.
- Un **dialecto**: SQLAlchemy puede trabajar con muchos tipos de bases de datos, siendo cada uno de estos tipos un dialecto diferente (MySQL, PostgreSQL, Oracle, SQLite, etcétera).

En nuestro caso el dialecto será SQLite y la ingesta de datos se hará en la base de datos database.sqlite3, por lo que instanciamos el *engine* de la siguiente manera:

In [ ]:
# echo flag logs the SQL queries executed by the engine. It’s helpful for
# debugging purposes (True), but don’t use it in a production environmen (False)
engine = create_engine('sqlite:///database.sqlite3', echo=True)

## 2) Ingesta de datos

### Lectura de datos

#### Datos de la competencia Kaggle

Leemos los datos de la competencia Kaggle utilizando pandas. Vemos que en total consta de 13.580 registros con respuestas a 21 variables diferentes.

In [ ]:
# Lectura del csv
url_kag = 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv'
melb_df = pd.read_csv(url_kag)
total_ans_kag = len(melb_df)
print(f'Cantidad de respuestas en el dataset de Kaggle: {total_ans_kag}')
display(melb_df[:3])

In [ ]:
print('--- Información disponible en el dataset de Kaggle ---')
keys_kag = melb_df.keys()
print(f'Contiene un total de {len(keys_kag)} columnas:')
for k in range(len(keys_kag)):
    print(f'{k+1}) {keys_kag[k]}')

#### Datos de Airbnb

Leemos los datos de Airbnb utilizando pandas. Vemos que en total consta de 22.895 registros con respuestas a 84 variables diferentes, *i.e.* tiene 9.315 registros más que en el dataset de Kaggle y responde a 63 variables más.

In [ ]:
# Lectura del csv
url_air = 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv'
airbnb_df = pd.read_csv(url_air)
total_ans_air = len(airbnb_df)
print(f'Cantidad de respuestas en el dataset de Airbnb: {total_ans_air}')
display(airbnb_df[:3])

In [ ]:
print('--- Información disponible en el dataset de Airbnb ---')
keys_air = airbnb_df.keys()
print(f'Contiene un total de {len(keys_air)} columnas:')
for k in range(len(keys_air)):
    print(f'{k+1}) {keys_air[k]}')

Vamos a reducir el dataframe y quedarnos sólo con aquellas columnas que consideramos relevantes para el análisis que pretendemos hacer. Coincide que son 21 variables, pero no necesariamente la relación es 1:1 con las variables de Kaggle.

In [ ]:
int_cols_air = [
    'host_location', 'host_neighborhood', 'street', 'neighborhood', 'city',
    'suburb', 'state', 'zipcode', 'latitude', 'longitude', 'is_location_exact',
    'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
    'beds', 'bed_type', 'price', 'weekly_price', 'monthly_price'
]

airbnb_df = airbnb_df[int_cols_air]
display(airbnb_df[:3])

### Procesamiento de códigos postales

Queremos combinar los datos de Airbnb con los datos de Kaggle. Para ello utilizaremos el código postal como clave común: 'Postcode' en melb_df y 'zipcode' en airbnb_df. Antes que anda, debemos asegurarnos que las columnas se encuentren limpias y con un formato común.

Por un lado, vemos que 'Postcode' en melb_df tiene un formato común: son todos float con un 1 decimal. Vamos a pasarlos todos a enteros.

In [ ]:
print('Formato original de los datos:')
display(melb_df['Postcode'].value_counts().iloc[:10])
print('---------------------------------------------------------------------\n')
melb_df['postcode_int'] = melb_df['Postcode'].fillna(0).astype('int')
print('Datos pasados a enteros:')
display(melb_df['postcode_int'].value_counts().iloc[:10])

Por otra parte , vemos que 'zipcode' en airbnb_df tiene uan mezcla de formatos: algunos son float con un 1 decimal y otros son enteros. Vamos a pasarlos todos a enteros.

In [ ]:
print('Formato original de los datos:')
display(airbnb_df['zipcode'].value_counts()[:10])
print('---------------------------------------------------------------------\n')
# Se estandariza el tipo de datos para la columna zipcode
airbnb_df['zipcode'] = pd.to_numeric(airbnb_df.zipcode, errors='coerce')
airbnb_df['zipcode_int'] = airbnb_df['zipcode'].fillna(0).astype('int')
print('Datos pasados a enteros:')
display(airbnb_df['zipcode_int'].value_counts()[:10])


### Ingesta

Transcribimos todos los registros de melb_df a la tabla "Kaggle" de la base de datos SQL creada previamente.

In [ ]:
melb_df.to_sql('kaggle', con=engine, if_exists="replace")

Transcribimos todos los registros de airbnb_df a la tabla "airbnb" de la base de datos SQL creada previamente.

In [ ]:
airbnb_df.to_sql('airbnb', con=engine, if_exists="replace")

## 3) Consultas

A la hora de ejecutar consultas SQL, no debemos hacerlo directamente con el *engine*, sino que debemos hacerlo a través de instancia de conexión, la cual se crea con `engine.connect()`, apartádose momentáneamente del *connection pool*. 

En términos de desempeño, es muy importante cerrar las conexiones luego de utilizarlas. Al cerrar la conexión (`.close()`), la conexión se recicla y vuelve al *pool* para ser reutilizada, lo cual previene el overhead de creación. Una alternativa al cierre de la conexión para su reciclaje es utilizar un bloque `with()`: la conexión se cerrará automáticamente luego de que se ejecute el bloque de código dentro del `with()`.

Otra cosa a tener en cuenta es que alimentar la ejecución con una *string* pelada está deprecado, debiendo usar en su lugar la función `text()` dentro de sqlalchemy para proveer la consulta SQL.

Considerando todo lo dicho, definimos una función para facilitar la ejecución de consultas.

**Observación:** la consigna pide distinguir por ciudad y por ciudad+barrio. Estos datos sólo están disponibles en la base de datos de Airbnb, así que las consultas se realizarán sobre la tabla correspondiente.

In [ ]:
def execute_query(query):
  with engine.connect() as con:
    rs = con.execute(text(query))
    df_rs = pd.DataFrame(rs.fetchall())
  return df_rs

### Cantidad de registros totales por ciudad

Veamos la cantidad de registros totales por ciudad, agrupando por la columna *city*. Ordenamos además de mayor a menor cantidad de registros. Tenemos que Melbourne va a la cabeza con 7368, seguido por Port Phillip con 2808.

In [ ]:
query_city = """SELECT city AS Ciudad, COUNT(*) AS Registros
                FROM airbnb
                GROUP BY city
                ORDER BY Registros DESC"""
df_city = execute_query(query_city)
df_city.head(10)

### Cantidad de registros totales por barrio y ciudad

Veamos ahora la cantidad de registros totales por barrio y por ciudad, agrupando por las columnas *neighborhood* y *city*. Ordenamos además de mayor a menor cantidad de registros. Al considerar los barrios, tenemos que Melbourne sigue encabezando la lista, siendo Central Business District y Southbank los dos mayoritarios, con 3726 y 1204 registros, respectivamente

va a la cabeza con 7368, seguido por Port Phillip con 2808.

In [ ]:
query_neighborhood_city = """SELECT neighborhood AS Barrio, city AS Ciudad, COUNT(*) AS Registros
                FROM airbnb
                GROUP BY neighborhood, city
                ORDER BY Registros DESC"""
df_neighborhood_city = execute_query(query_neighborhood_city)
df_neighborhood_city.head(10)

## 4) Combinación de tablas

Considerando que nuestro objetivo es combinar ambas tablas, ampliando así la información disponible en el dataset de Kaggle sobre el valor del vecindario de la propiedad con información proveniente del dataset de AirBnB, utilizaremos de esta última tabla lo siguientes promedios asociados al código postal:
* Promedio del precio diario (`price`).
* Promedio del precio semanal (`weekly_price`).
* Promedio del precio mensual (`monthly_price`).

Consideremos además el conteo de cuántos registros responden a cada uno de estos promedios, para así tener una manera de ponderar la relevancia que nos aporta cada uno de estos datos a la hora de extraer conclusiones.

Para lograr esto usando el comando `JOIN` de SQL, ejecutamos una subconsulta, donde se define una tabla `airbnb_agg` con todas las agregaciones deseadas. Finalmente, esta nueva tabla es combinada con la tabla de Kaggle considerando los códigos postales.

In [ ]:
query_join = """WITH airbnb_agg AS (
                  SELECT zipcode_int, 
                    AVG(price) AS airbnb_daily_mean,
                    COUNT(price) AS airbnb_daily_count,
                    AVG(weekly_price) AS airbnb_weekly_mean,
                    COUNT(weekly_price) AS airbnb_weekly_count,
                    AVG(monthly_price) AS airbnb_monthly_mean,
                    COUNT(monthly_price) AS airbnb_monthly_count
                  FROM airbnb
                  GROUP BY zipcode_int
                )
                SELECT * FROM Kaggle A
                LEFT JOIN airbnb_agg B 
                ON A.Postcode_int = B.zipcode_int"""
df_join = execute_query(query_join)

In [ ]:
df_join.sample(10)

---
# Ejercicio 2 - Pandas

## 1) Selección de columnas relevantes

### Introducción

Retomamos el dataset de Kaggle ya cargado con pandas (`melb_d`) y generamos uno nuevo (`df`) donde nos quedamos con la columna de código postal ya procesada. El objetivo final es filtrar las columnas, seleccionando aquellas que consideramos relevantes para una buena predicción del valor de la propiedad.

Comenzaremos nuestro análisis separando las 21 columnas entre categóricas y numéricas. De las 13 variables numéricas, 5 hace referencia a la ubicación geográfica (Distance, Lattitude, Longtitude, Propertycount y Postcode) mientras que las otras 8 son características de la vivienda (Rooms, Price,  Bedroom2, Bathroom, Car, Landsize, BuildingArea y YearBuilt). Por otro lado, entre las 8 categóricas tenemos 4 que hacen referencia a la ubicación geográfica (Suburb, Address, CouncilArea y Regionname) y 4 que caracterizan de alguna manera la vivienda (Method, SellerG, Date y Type). Tendremos esto en cuenta a la hora de encarar los diferentes análisis.

In [ ]:
# Removemos la columna 'Postcode' ya que tenemos la columna editada 'postcode_int'
df = melb_df.copy()
df = df.drop(['Postcode'], axis=1)
df.rename(columns = {'postcode_int': 'Postcode'}, inplace = True)

num_geo = ['Distance', 'Propertycount',
            'Lattitude', 'Longtitude']
num_home = ['YearBuilt', 'Landsize', 'BuildingArea', 'Price',
            'Rooms', 'Bedroom2', 'Bathroom', 'Car']

cat_geo = ['Suburb', 'Address', 'CouncilArea', 'Regionname', 'Postcode']
cat_home = ['Method', 'SellerG', 'Date', 'Type']

### Análisis de variables numéricas

#### Filtrado de variables numéricas geográficas

Tomando conjuntamente la estadística descriptiva junto a los histogramas, vemos que:
* Tanto longitud y latitud están bastante simétricamente distribuidas.
* Las otras 3 variables son asimétricas hacia la derecha.

En el caso de Propertycount se observa un valor extremo mayor a 20.000, sin embargo este valor posee una cantidad importante de cuentas (aproximadamente un tercio del máximo) por lo que no lo descartamos.

In [ ]:
df[num_geo].describe()

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(12, 10))    

sns.histplot(df[num_geo[0]], ax=axs[0, 0], color='tab:orange')

sns.histplot(df[num_geo[1]], ax=axs[0, 1], color='tab:orange')

sns.histplot(df[num_geo[2]], ax=axs[1, 0], color='tab:orange')

sns.histplot(df[num_geo[3]], ax=axs[1, 1], color='tab:orange')

axs[0, 0].set_ylabel("")
axs[0, 1].set_ylabel("")
axs[1, 0].set_ylabel("")
axs[1, 1].set_ylabel("")

plt.show()

#### Filtrado de variables numéricas que caracterizan a la vivienda

Tomando conjuntamente la estadística descriptiva junto a los histogramas, vemos que todas las distribuciones son asimétricas. Particularmente:
* YearBuilt tiene cola hacia la izquierda.
* Las demás tienen cola hacia la derecha.

Considerando las colas, en principio creemos necesario cortar:
* Bedroom2 menor a 20
* Landsize menor a 3000
* BuildingArea menor a 1500
* YearBuilt mayor a 1200
* Price menor a 6000000

De esta forma eliminamos los valores extremos que se observa en cada uno de las variables numéricas que caracterizan a la vivienda.

En el proceso se eliminaron 147 registros. Con esto se ha quitado el 1.08% del total de registros originales.

In [ ]:
df[num_home].describe()

In [ ]:
fig, axs = plt.subplots(2,4, figsize=(24, 10))    

sns.histplot(df[num_home[0]], ax=axs[0, 0], color='tab:orange')
sns.histplot(df[num_home[1]], ax=axs[0, 1], color='tab:orange')
sns.histplot(df[num_home[2]], ax=axs[1, 0], color='tab:orange')
sns.histplot(df[num_home[3]], ax=axs[1, 1], color='tab:orange')
sns.histplot(df[num_home[4]], ax=axs[0, 2], color='tab:orange')
sns.histplot(df[num_home[5]], ax=axs[0, 3], color='tab:orange')
sns.histplot(df[num_home[6]], ax=axs[1, 2], color='tab:orange')
sns.histplot(df[num_home[7]], ax=axs[1, 3], color='tab:orange')

axs[0, 0].set_ylabel("")
axs[0, 1].set_ylabel("")
axs[1, 0].set_ylabel("")
axs[1, 1].set_ylabel("")
axs[0, 2].set_ylabel("")
axs[0, 3].set_ylabel("")
axs[1, 2].set_ylabel("")
axs[1, 3].set_ylabel("")

plt.show()

In [ ]:
# En todos los casos se conservan los nulos para luego ser imputados
df_filtered = df.copy()
df_filtered = df_filtered[(df_filtered['Bedroom2'] < 20) | (df_filtered['Bedroom2'].isnull())]
df_filtered = df_filtered[(df_filtered['Landsize'] < 3000) | (df_filtered['Landsize'].isnull())]
df_filtered = df_filtered[(df_filtered['BuildingArea'] < 1500)  | (df_filtered['BuildingArea'].isnull())]
df_filtered = df_filtered[(df_filtered['YearBuilt'] > 1200) | (df_filtered['YearBuilt'].isnull())]
df_filtered = df_filtered[(df_filtered['Price'] < 6000000) | (df_filtered['Price'].isnull())]

In [ ]:
fig, axs = plt.subplots(2,4, figsize=(24, 10))    

sns.histplot(df_filtered[num_home[0]], ax=axs[0, 0], color='tab:orange')
sns.histplot(df_filtered[num_home[1]], ax=axs[0, 1], color='tab:orange')
sns.histplot(df_filtered[num_home[2]], ax=axs[1, 0], color='tab:orange')
sns.histplot(df_filtered[num_home[3]], ax=axs[1, 1], color='tab:orange')
sns.histplot(df_filtered[num_home[4]], ax=axs[0, 2], color='tab:orange')
sns.histplot(df_filtered[num_home[5]], ax=axs[0, 3], color='tab:orange')
sns.histplot(df_filtered[num_home[6]], ax=axs[1, 2], color='tab:orange')
sns.histplot(df_filtered[num_home[7]], ax=axs[1, 3], color='tab:orange')

axs[0, 0].set_ylabel("")
axs[0, 1].set_ylabel("")
axs[1, 0].set_ylabel("")
axs[1, 1].set_ylabel("")
axs[0, 2].set_ylabel("")
axs[0, 3].set_ylabel("")
axs[1, 2].set_ylabel("")
axs[1, 3].set_ylabel("")

plt.show()

In [ ]:
print(f'\tEliminamos {len(df) - len(df_filtered)} registros.')
print(f'\t{100*(len(df) - len(df_filtered))/len(df):.2f}% del total original.\n')

#### Análisis de correlación entre variables numéricas geográficas y el precio

Determinamos la correlación entre variables numéricas geográficas y el precio tanto analítica como gráficamente. A partir de esto, tomamos las siguientes decisiones:
* Distance: si bien la correlación con el precio es baja (-0.17), se puede observar que a medida que la distancia al distrito comercial central crece, los precios disminuyen. Particularmente, los precios más bajos se encuentran a distancias mayores a 20. Por esta razón la consideramos en el dataset.
* Propertycount: la correlación con el precio es de -0.04 y no hay una tendencia clara en las gráficas, por lo que la descartamos del dataset.
* Lattitude y Longitude: en ambos casos la correlación con el precio es similar en intensidad (-0.22 y 0.20, respectivamente). Al considerar sus gráficas, hay una clara acumulación de puntos, definiendo una tendencia entre la ubiación geográfica y el precio, como era de esperar. Consideraremos ambas en el dataset.

In [ ]:
# Cálculo de coeficientes de correlación con el precio
corr = df_filtered[num_geo+['Price']].corr()['Price']
print(corr.drop('Price'))

# Gráfica de correlaciones con el precio
for col in num_geo:
    sns.jointplot(data=df_filtered, y='Price', x=col, s=10, alpha=0.01, color='tab:green', height=3)
    plt.grid()
plt.show()

#### Análisis de correlación entre variables numéricas que caracterizan la vivienda y el precio

Determinamos la correlación entre variables numéricas que caracterizan la vivienda y el precio tanto analítica como gráficamente. A partir de esto, tomamos las siguientes decisiones:
* YearBuilt: tanto la correlación con el precio (-0.33) como la gráfica nos dan una tendencia clara de que cuanto más antigüa es la vivienda, mayor es su precio, concentrándose los menores precios en las más recientes. La consideramos en el dataset.
* Landsize: si bien la correlación con el precio es baja (0.25), no debemos dejar de lado que es una correlación lineal. Al ver la gráfica, tenemos claramente dos *poblaciones* bien marcadas para el tamaño de la propiedad. Además, éstas se concentran para precios bajos. Consideramos que es una variable importante para predecir el valor de la propiedad y la consideramos en el dataset.
* BuildingArea: tiene una mayor correlación que Landsize (0.54), lo cual tiene sentido al tener una única *población* en su gráfica. Hay una marcada tendencia entre tamaños y precios, por lo que la consideramos en el dataset. 
* Rooms: la correlación con el precio es de 0.50 y esto se ve reflejado en su gráfica, por lo que consideramos que es importante para la predicción del valor de la propiedad.
* Bedroom2: es totalmente análogo el caso Rooms, por lo que la consideraremos en el dataset.
* Bathroom: pasa algo similar a los 2 casos anteriores, aunque no es tan marcado. La consideraremos en el dataset.
* Car: la correlación con el precio es de 0.24 y su gráfica no indica una tendencia muy clara, por lo que la descartamos.

In [ ]:
# Cálculo de coeficientes de correlación con el precio
corr = df_filtered[num_home].corr()['Price']
print(corr.drop('Price'))

# Gráfica de correlaciones con el precio
for col in num_home:
    if col == 'Price':
        continue
    else:
        sns.jointplot(data=df_filtered, y='Price', x=col, s=10, alpha=0.01, color='tab:green', height=3)
plt.show()

#### Variables numéricas elegidas

In [ ]:
# Nuevo listado de variables numéricas de interés
sel_num = ['Distance', 'Lattitude', 'Longtitude', 
            'YearBuilt', 'Landsize', 'BuildingArea', 'Rooms', 
            'Bedroom2', 'Bathroom', 'Price']

### Análisis de variables categóricas

Para decidir sobre este tipo de variables, consideraremos conjuntamente la cardinalidad de cada variable y el porcentaje de repeticiones que tienen los 10 registros mayoritarios.

#### Filtrado de variables categóricas geográficas

Suburb tiene 311 valores diferentes. El valor mayoritario contribuye en un 2.66% y los 10 primeros en conjunto contribuyen al 17.09%. Descartamos entonces esta variable del dataset.

In [ ]:
print('Cardinalidad:', df_filtered[cat_geo[0]].nunique())
a = 100*df_filtered[cat_geo[0]].value_counts(normalize=True).iloc[:10]
print(a)
print(np.sum(a))

Address tiene 13232 valores diferentes. El valor mayoritario contribuye en un 0.02% y los 10 primeros en conjunto contribuyen al 0.22%. Descartamos entonces esta variable del dataset.

**Observación:** más allá de estos números, descartamos esta variable ya que la dirección es única para cada propiedad y no es representativa.

In [ ]:
print('Cardinalidad:', df_filtered[cat_geo[1]].nunique())
a = 100*df_filtered[cat_geo[1]].value_counts(normalize=True).iloc[:10]
print(a)
print(np.sum(a))

CouncilArea tiene 33 valores diferentes. El valor mayoritario contribuye en un 9.59% y los 10 primeros en conjunto contribuyen al 68.61%. Esta variable no será descartada del dataset.

In [ ]:
print('Cardinalidad:', df_filtered[cat_geo[2]].nunique())
a = 100*df_filtered[cat_geo[2]].value_counts(normalize=True).iloc[:10]
print(a)
print(np.sum(a))

Regionname tiene 8 valores diferentes. El valor mayoritario contribuye en un 34.54% y los 10 primeros en conjunto contribuyen al 100%. Esta variable no será descartada del dataset.

In [ ]:
print('Cardinalidad:', df_filtered[cat_geo[3]].nunique())
a = 100*df_filtered[cat_geo[3]].value_counts(normalize=True).iloc[:10]
print(a)
print(np.sum(a))

Postcode tiene 196 valores diferentes. El valor mayoritario contribuye en un 2.65% y los 10 primeros en conjunto contribuyen al 20.32%. La descartaríamos, pero la conservamos ya que será nuestro punto de unión entre datasets.

In [ ]:
print('Cardinalidad:', df_filtered[cat_geo[4]].nunique())
a = 100*df_filtered[cat_geo[4]].value_counts(normalize=True).iloc[:10]
print(a)
print(np.sum(a))

#### Filtrado de variables categóricas que caracterizan a la vivienda

Method tiene 5 valores diferentes. El valor mayoritario contribuye en un 66.52% y estos 5 contribuyen al 100%. Esta variable no será descartada del dataset.

In [ ]:
print('Cardinalidad:', df_filtered[cat_home[0]].nunique())
a = 100*df_filtered[cat_home[0]].value_counts(normalize=True).iloc[:10]
print(a)
print(np.sum(a))

SellerG tiene 267 valores diferentes. El valor mayoritario contribuye en un 11.53% y los 10 primeros en conjunto contribuyen al 59.53%. Esta variable no será descartada del dataset.

In [ ]:
print('Cardinalidad:', df_filtered[cat_home[1]].nunique())
a = 100*df_filtered[cat_home[1]].value_counts(normalize=True).iloc[:10]
print(a)
print(np.sum(a))

Date tiene 58 valores diferentes. El valor mayoritario contribuye en un 3.48% y los 10 primeros en conjunto contribuyen al 26.77%. Descartamos entonces esta variable del dataset.

In [ ]:
print('Cardinalidad:', df_filtered[cat_home[2]].nunique())
a = 100*df_filtered[cat_home[2]].value_counts(normalize=True).iloc[:10]
print(a)
print(np.sum(a))

Type tiene 3 valores diferentes. El valor mayoritario contribuye en un 70.04% y estos 3 contribuyen al 100%. Esta variable no será descartada del dataset.

In [ ]:
print('Cardinalidad:', df_filtered[cat_home[3]].nunique())
a = 100*df_filtered[cat_home[3]].value_counts(normalize=True).iloc[:10]
print(a)
print(np.sum(a))

#### Variables categóricas elegidas

En el caso de las variables de mayor cardinalidad (CouncilArea y SellerG), agruparemos todos los registros que que contribuyen en menos del 2% en un nuevo registro "Other". De esta manera pasamos de tener 220 valores únicos para SellerG y 28 para CouncilArea, a tener 13 y 18 valores únicos, respectivamente

In [ ]:
for col in ['SellerG', 'CouncilArea']:
  value_counts = 100 * df_filtered[col].value_counts(normalize=True)
  lower_values = value_counts[value_counts < 2].index.tolist()
  df_filtered[col] = df_filtered[col].apply(lambda x: 'Other' if x in lower_values else x)

In [ ]:
for col in ['SellerG', 'CouncilArea']:
    print(col)
    print('\tCardinalidad:', df_filtered[col].nunique())

In [ ]:
# Preselección de variables categóricas de interés
sel_cat = ['CouncilArea', 'Regionname', 'Method', 'SellerG', 'Type', 'Postcode']

Ahora veamos los boxplots para las variables categóricas preseleccionadas. Salvo para Method, en todos los demás casos vemos que se pueden establecer tendencias entre el valor de la variable categórica y el precio, por lo que nos quedamos con CouncilArea, Regionname, SellerG y Type.

In [ ]:
for col in sel_cat[:-1]:
  values = df_filtered[col].value_counts(normalize=True).index[:10].tolist()
  plt.figure(figsize=(6, 5))
  sns.boxplot(data=df_filtered[df_filtered[col].isin(values)], y=col, x='Price')
  plt.show()

In [ ]:
# Nuevo listado de variables categóricas de interés
sel_cat = ['CouncilArea', 'Regionname', 'SellerG', 'Type', 'Postcode']

### Filtrado del DataFrame por variables seleccionadas

Creamos un nuevo DataFrame donde nos quedamos únicamente con las columnas de interés, ya teniendo ciertos filtros aplicados. Pasamos de tener 21 variables y 13.580 registros a tener 15 variables y 9.418 registros.

In [ ]:
sel_cols = sel_num + sel_cat
print(f'Cantidad de variables: {len(sel_cols)}.')

df = df_filtered.copy()
df = df[sel_cols]
print(f'Cantidad de registros: {len(df)}.')

df.head()

## 2) Unión de datasets

In [ ]:
airbnb_df = pd.read_csv('https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv')

In [ ]:
# Se estandariza el tipo de datos para la columna zipcode
airbnb_df['zipcode'] = pd.to_numeric(airbnb_df.zipcode, errors='coerce')

In [ ]:
airbnb_df.head(3)

In [ ]:
airbnb_df.zipcode.value_counts()

In [ ]:
# Se realiza un scatter plot para apreciar cuántas entradas hay para cada zipcode
sns.scatterplot(data=airbnb_df.zipcode.value_counts().values) # Agregué 'data=' para que funcione sns
plt.axhline(100, color='g')
plt.xlabel("Índice (zipcodes ordenados de mayor a menor en #repeticiones)")
plt.ylabel("#repeticiones")
plt.show()

In [ ]:
len(airbnb_df.zipcode.value_counts()[airbnb_df.zipcode.value_counts()<100])

De aquí en adelante solo consideraremos los zipcodes que tengan una cantidad mayor o igual a 100 registros para que la información agregada sea relevante. De acuerdo al análisis anterior, esto implica eliminar 199 zipcodes. Destacamos que el zipcode más abundante es "3000", el cual aparece 3367 veces.

In [ ]:
value_counts = airbnb_df.zipcode.value_counts()
value_counts = value_counts[value_counts>=100]
airbnb_df_reducido = airbnb_df.copy()  # Copio el df para que no se pierda el anterior
airbnb_df_reducido = airbnb_df_reducido[airbnb_df_reducido['zipcode'].isin(value_counts.index.tolist())]

In [ ]:
airbnb_price_by_zipcode = airbnb_df_reducido.groupby('zipcode')\
  .agg({'price': ['mean', 'count'], 'weekly_price': 'mean',
        'monthly_price': 'mean', 'number_of_reviews': ['sum', 'mean'],
        'review_scores_rating': ['min', 'max', 'mean']})\
  .reset_index()
# Flatten the two level columns
airbnb_price_by_zipcode.columns = [
  ' '.join(col).strip()
  for col in airbnb_price_by_zipcode.columns.values]

In [ ]:
airbnb_price_by_zipcode.columns

In [ ]:
# Rename columns
airbnb_price_by_zipcode = airbnb_price_by_zipcode.rename(
    columns={'price mean': 'airbnb_price_mean',
             'price count': 'airbnb_record_count',
             'weekly_price mean': 'airbnb_weekly_price_mean',
             'monthly_price mean': 'airbnb_monthly_price_mean',
             'number_of_reviews sum': 'airbnb_number_of_reviews_sum', 
             'number_of_reviews mean': 'airbnb_number_of_reviews_mean', 
             'review_scores_rating min': 'airbnb_review_scores_rating_min', 
             'review_scores_rating max': 'airbnb_review_scores_rating_max',
             'review_scores_rating mean': 'airbnb_review_scores_rating_mean'}
)

In [ ]:
airbnb_price_by_zipcode.head()

In [ ]:
df_join = df.merge(
    airbnb_price_by_zipcode, how='left',
    left_on='Postcode', right_on='zipcode'
)
df_join.sample(5)

## 3) Otras posibilidades

Se podría utilizar la variable Suburb que se encuentra en ambos datasets y posee una cardinalidad mayor al código postal en ambos casos

In [ ]:
print('Cardinalidad Código postal:', melb_df.Postcode.nunique())
print('Cardinalidad Suburbio:', melb_df.Suburb.nunique())

In [ ]:
print('Cardinalidad Código postal:', airbnb_df.zipcode.nunique())
print('Cardinalidad Suburbio:', airbnb_df.suburb.nunique())

Otra opción sería utilizar la latitud y longitud que se encuentra en ambos datasets y encontrar todas las propiedades que se encuentren a cierta distancia.

---
# Ejercicio 3 - Guardado final

In [ ]:
df_join.head()

Guardamos el dataset en un archivo csv:

In [ ]:
import os  # Para crear el archivo csv en el cwd sin tener que usar colab.
df_join.to_csv('./GuardadoFinal.csv')